# Land cover categories and change in NO2

In [1]:
import os
import numpy as np
import ee
import geemap.foliumap as geemap
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

ee.Initialize()

Landsat 8 surface reflectance data with least cloud cover in the given period

In [128]:
def get_landsat_images(location, start_date, end_date):
    image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(location)
    .filterDate(start_date, end_date)
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
    )
    
    print(ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo())
    print(image.get('CLOUD_COVER').getInfo())
    return image

Coordinates of Los Angeles

In [129]:
point = ee.Geometry.Point([-118.2305, 34.0693])

National Land Cover Database and corresponding Landsat 8 image 

In [130]:
nlcd_2016 = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL').filter(ee.Filter.eq('system:index', '2016'))
nlcd = nlcd_2016.select('landcover').first()

nlcd_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
metadata = nlcd_metadata.filterBounds(point).first()
region = metadata.geometry()

doy = metadata.get('2016on_bas').getInfo().replace('LC08_', '')
start_nlcd = ee.Date.parse('YYYYDDD', doy)
end_nlcd = start_nlcd.advance(1, 'day')

nlcd_landsat = get_landsat_images(point, start_nlcd, end_nlcd).clip(region)
nlcd_raw = nlcd.clip(region)

2016-09-26
0.01


NLCD maps

In [131]:
m = geemap.Map(center=(34.57, -118.23), zoom=8)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

m.addLayer(nlcd_raw, {}, 'NLCD LA')
m.addLayer(nlcd_landsat, vis_params, 'NLCD Landsat-8 LA')

Regrouping land cover categories into 5 classes.

1 : Water

2 : Urban     

3 : Forest    

4 : Vegetation   

5 : Other

In [132]:
raw_class_values = nlcd_raw.get('landcover_class_values').getInfo()
new_class_values = [1,1,2,2,2,2,5,3,3,3,4,4,4,4,4,4,4,4,5,5] 

class_palette = nlcd_raw.get('landcover_class_palette').getInfo()
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(['remapped'], ['landcover'])
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

m.addLayer(nlcd, {}, 'NLCD remapped')

Training and validation data (80-20 split over 5000 training pixels)

In [133]:
landsat_scale = nlcd_landsat.projection().nominalScale().getInfo()
points = nlcd.sample(region=region, scale=landsat_scale, numPixels=5000,
                     seed=0, geometries=True)
m.addLayer(points, {}, 'training')

bands = nlcd_landsat.bandNames().getInfo()
label = 'landcover'

sample = nlcd_landsat.select(bands).sampleRegions(collection=points, properties=[label], scale=landsat_scale)

sample = sample.randomColumn()
split = 0.8

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

Random Forest classifier

In [134]:
classifier = ee.Classifier.smileRandomForest(128).train(training, label, bands)

train_accuracy = classifier.confusionMatrix()
train_accuracy.accuracy().getInfo()

0.990071283095723

In [135]:
fig = px.imshow(train_accuracy.getInfo(), text_auto=True, 
                labels=dict(x="Predicted", y="Actual"))
fig.show()

Cross-validation accuracy

In [136]:
validated = validation.classify(classifier)
test_accuracy = validated.errorMatrix('landcover', 'classification')
test_accuracy.accuracy().getInfo()

0.872057318321392

In [137]:
fig = px.imshow(test_accuracy.getInfo(), text_auto=True, 
                labels=dict(x="Predicted", y="Actual"))
fig.show()

Test images : Landsat 8 image for 2019 and 2020

In [138]:
landsat_2019 = get_landsat_images(point, '2019-01-01', '2019-04-01')
landsat_2020 = get_landsat_images(point, '2020-01-01', '2020-04-01')

m.addLayer(landsat_2019, vis_params, "Landsat-8 2019")
m.addLayer(landsat_2020, vis_params, "Landsat-8 2020")

2019-01-22
1.19
2020-02-26
0.17


In [139]:
result_2019 = landsat_2019.select(bands).classify(classifier)
result_2020 = landsat_2020.select(bands).classify(classifier)

In [140]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_palette = nlcd.get('landcover_class_palette').getInfo()

landcover_2019 = result_2019.set('classification_class_values', class_values)
landcover_2019 = landcover_2019.set('classification_class_palette', class_palette)

landcover_2020 = result_2020.set('classification_class_values', class_values)
landcover_2020 = landcover_2020.set('classification_class_palette', class_palette)

Interactive map

In [141]:
m.addLayer(landcover_2019, {}, 'Land cover 2019')
m.addLayer(landcover_2020, {}, 'Land cover 2020')

m.addLayerControl()
m.to_html(filename='results/fig4_5.html', title='NO2 diff Map', width='100%', height='880px')
m

NO2 maps for 2019 and 2020 for the period corresponding to lockdown

In [142]:
def get_NO2_map(start_date, end_date):
    NO2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(start_date, end_date)
    return NO2

roi = landsat_2019.geometry()

NO2_2019 = get_NO2_map('2019-03-25', '2019-05-03').mean()
NO2_2020 = get_NO2_map('2020-03-25', '2020-05-03').mean()

palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
vis_params = {'min': 0, 'max': 0.00015, 'palette': palette}
vis_params_diff = {'min': -0.00001, 'max': 0.0001, 'palette': palette}

m.addLayer(NO2_2019.clip(roi), vis_params, 'NO2 2019')
m.addLayer(NO2_2020.clip(roi), vis_params, 'NO2 2020')

Pixels corresponding to a given class in 2019 and 2020 and average NO2 values for each year

In [143]:
def get_change_NO2(class_value, name):
    class_2019 = landcover_2019.eq(class_value)
    class_2020 = landcover_2020.eq(class_value)

    NO2_class_2019 =  NO2_2019.mask(class_2019)
    NO2_class_2020 =  NO2_2020.mask(class_2020)

    m.addLayer(NO2_class_2019, vis_params, name+'NO2 2019')
    m.addLayer(NO2_class_2020, vis_params, name+'NO2 2020')

    class_avg_2019 = NO2_class_2019.reduceRegion(ee.Reducer.mean(), geometry=roi, scale=landsat_scale, maxPixels=1e9).getInfo()
    class_avg_2020 = NO2_class_2020.reduceRegion(ee.Reducer.mean(), geometry=roi, scale=landsat_scale, maxPixels=1e9).getInfo()

    band_name = 'tropospheric_NO2_column_number_density'
    class_NO2_diff = class_avg_2020[band_name] - class_avg_2019[band_name]

    return class_NO2_diff

Classwise masks and change in NO2

In [144]:
urban_NO2_diff = get_change_NO2(2, 'urban')
forest_NO2_diff = get_change_NO2(3, 'forest')
veg_NO2_diff = get_change_NO2(4, 'vegetation')

In [145]:
fig = px.bar(x=['urban', 'forest', 'vegetation'], y=[urban_NO2_diff, forest_NO2_diff, veg_NO2_diff])
fig.update_layout(xaxis_title='', yaxis_title='Reduction in NO2 (mol/m^2)')
fig.write_html('results/fig6.html')
fig.show()

Maximum reduction in NO2 is observed over urban areas